In [8]:
import os
import sys
sys.path.append('/home/guangp/BirdCLEF-2025_Melspec')

import warnings
warnings.filterwarnings('ignore')

os.environ["CUDA_VISIBLE_DEVICES"] = "1,3"

In [9]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, BackboneFinetuning, EarlyStopping

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

import importlib
from pathlib import Path
import numpy as np
import glob
import timm
import pandas as pd
import torchaudio as ta

from modules.preprocess import preprocess,prepare_cfg
from modules.dataset import get_train_dataloader
from modules.model import load_model
import modules.inception_next_nano

In [10]:
# move to repo root
cur_dir = Path().resolve()

if not (cur_dir / "notebooks").exists():
    os.chdir(os.path.abspath("../"))
print(f"{Path().resolve()}")

/home/guangp/BirdCLEF-2025_Melspec


In [11]:
model_name = "cnn_v1"
stage = "train_bce"

cfg = importlib.import_module(f'configs.{model_name}').basic_cfg
cfg = prepare_cfg(cfg,stage)

pl.seed_everything(cfg.seed[stage], workers=True)

Seed set to 20231911


20231911

In [12]:
df_train, df_valid, df_label_train, df_label_valid, transforms = preprocess(cfg, stage)
df_train["version"] = "2023"
df_valid["version"] = "2023"
len(df_train), len(df_valid)

(22851, 5713)

In [13]:
df_train

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,...,author,license,path,duration,sr,max_amplitude,duration_sec,weight,presence_type,version
0,yehcar1,[],['call'],yehcar1/XC674420.ogg,XC,5.0,https://xeno-canto.org/674420,-21.7357,-43.3693,Milvago chimachima,...,Marco Manhães,cc-by-nc-sa 4.0,/data1/Mamba/Kaggle/BirdCLEF-2025/Data_Raw/tra...,1453248,22050,1.005431,65.906939,0.330124,foreground,2023
1,rutjac1,[rumfly1],['song'],rutjac1/XC261831.ogg,XC,4.5,https://xeno-canto.org/261831,0.8830,-78.8000,Galbula ruficauda,...,Olaf Jahn,cc-by-nc-sa 4.0,/data1/Mamba/Kaggle/BirdCLEF-2025/Data_Raw/tra...,563904,22050,0.440904,25.573878,0.315243,foreground,2023
2,grekis,[],['song'],grekis/XC576073.ogg,XC,4.0,https://xeno-canto.org/576073,14.3572,-87.6545,Pitangus sulphuratus,...,Mario Reyes Jr,cc-by-nc-sa 4.0,/data1/Mamba/Kaggle/BirdCLEF-2025/Data_Raw/tra...,973440,22050,0.666072,44.146939,0.161863,foreground,2023
3,rutjac1,[],['song'],rutjac1/XC298078.ogg,XC,4.0,https://xeno-canto.org/298078,7.8273,-72.6175,Galbula ruficauda,...,Oscar Laverde,cc-by-nc-sa 4.0,/data1/Mamba/Kaggle/BirdCLEF-2025/Data_Raw/tra...,444673,22050,0.101076,20.166576,0.315243,foreground,2023
4,rumfly1,[],"['duet', 'song']",rumfly1/XC54522.ogg,XC,4.0,https://xeno-canto.org/54522,NaN,NaN,Myiozetetes cayanensis,...,Bernabe Lopez-Lanus,cc-by-nc-sa 3.0,/data1/Mamba/Kaggle/BirdCLEF-2025/Data_Raw/tra...,474625,22050,0.555774,21.524943,0.331520,foreground,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22846,linwoo1,[],['drumming'],linwoo1/XC78185.ogg,XC,3.0,https://xeno-canto.org/78185,-19.1824,-48.3936,Dryocopus lineatus,...,Bob Planque,cc-by-nc-sa 3.0,/data1/Mamba/Kaggle/BirdCLEF-2025/Data_Raw/tra...,221761,22050,0.652465,10.057188,0.280355,foreground,2023
22847,socfly1,[],[''],socfly1/iNat880781.ogg,iNat,0.0,https://static.inaturalist.org/sounds/880781.mp3,-18.8302,-40.6809,Myiozetetes similis,...,Vitor Corrêa Dias,cc-by-nc 4.0,/data1/Mamba/Kaggle/BirdCLEF-2025/Data_Raw/tra...,110603,22050,0.051320,5.016009,0.218558,foreground,2023
22848,greani1,[],['call'],greani1/XC14741.ogg,XC,3.0,https://xeno-canto.org/14741,-3.2231,-59.8664,Crotophaga major,...,Nick Athanas,cc-by-nc-sa 3.0,/data1/Mamba/Kaggle/BirdCLEF-2025/Data_Raw/tra...,793729,22050,0.865305,35.996780,0.451922,foreground,2023
22849,grekis,[],['song'],grekis/XC714034.ogg,XC,4.0,https://xeno-canto.org/714034,-23.7995,-46.0230,Pitangus sulphuratus,...,Fernando Igor de Godoy,cc-by-nc-sa 4.0,/data1/Mamba/Kaggle/BirdCLEF-2025/Data_Raw/tra...,1323648,22050,0.191671,60.029388,0.161863,foreground,2023


In [7]:
if cfg.use_2024_additional_cleaned:
    # append handlabeled data
    df_additional = pd.read_pickle(cfg.train_2024_additional_cleaned)
    df_additional["version"] = "2023"
    df_additional["presence_type"]  = "foreground"

    # repeating records
    df_additional = pd.concat([df_additional] * cfg.num_cleaned_repeat).reset_index(drop=True)

    # make one-hot label
    add_primary_label = pd.Categorical(df_additional["primary_label"], categories=cfg.bird_cols)
    add_primary_label = pd.get_dummies(add_primary_label,  dtype=np.float64)
    assert (add_primary_label.columns == df_label_train.columns).all()


    df_train = pd.concat([df_train, df_additional]).reset_index(drop=True)
    df_label_train = pd.concat([df_label_train, add_primary_label]).reset_index(drop=True)
    # df_train.shape

    # shuffle
    perm_idx = df_train.index.to_series().sample(frac=1, random_state=0)
    df_train = df_train.iloc[perm_idx].reset_index(drop=True)
    df_label_train = df_label_train.iloc[perm_idx].reset_index(drop=True)

    all_primary_labels = df_train["primary_label"]
    sample_weights = (
        all_primary_labels.value_counts() / 
        all_primary_labels.value_counts().sum()
    )  ** (cfg.class_exponent_weight)
    sample_weights = sample_weights / sample_weights.mean()
    df_train["weight"] = sample_weights[df_train["primary_label"].values].values

len(df_train)

22851

In [8]:
pseudo = None
dl_train, dl_val, ds_train, ds_val = get_train_dataloader\
    (
		df_train,
		df_valid,
		df_label_train,
		df_label_valid,
		cfg,
		pseudo,
		transforms
    )

In [9]:
# batch = next(iter(dl_train))
# audio, target, weight = batch
# audio.shape, target.shape

In [10]:
logger = WandbLogger(project='BirdClef-2025', name=f'{model_name}_{stage}')
checkpoint_callback = ModelCheckpoint\
    (
		monitor='val_loss',
		dirpath= cfg.output_path[stage],
		save_top_k=5,
		save_last=True,
		save_weights_only=True,
		filename='{epoch}_{step}_{val_roc_auc:.3f}_{val_cmap_pad:.3f}_{val_ap:.3f}',
		verbose=True,
		every_n_epochs=1,
		mode='min'
	)

## Trainer

In [11]:
logger = WandbLogger(project='BirdClef-2025', name=f'{model_name}_{stage}')
checkpoint_callback = ModelCheckpoint\
    (
		monitor='val_loss',
		dirpath= cfg.output_path[stage],
		save_top_k=5,
		save_last=True,
		save_weights_only=True,
		filename='{epoch}_{step}_{val_roc_auc:.3f}_{val_cmap_pad:.3f}_{val_ap:.3f}',
		verbose=True,
		every_n_epochs=1,
		mode='min'
	)

In [12]:
callbacks_to_use = [checkpoint_callback]
model = load_model(cfg,stage)
trainer = pl.Trainer\
    (
		devices=1,
		val_check_interval=1.0,
		deterministic=None,
		max_epochs=cfg.epochs[stage],
		logger=logger,
		callbacks=callbacks_to_use,
		precision=cfg.PRECISION, 
		accelerator="auto",
	)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [13]:
trainer.fit(model, train_dataloaders=dl_train, val_dataloaders=dl_val)

You are using a CUDA device ('NVIDIA A800 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Currently logged in as: liguangpeng9495 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1,3]

   | Name                | Type              | Params | Mode 
-------------------------------------------------------------------
0  | loss_function       | BCEWithLogitsLoss | 0      | train
1  | mixup               | Mixup             | 0      | train
2  | mixup2              | Mixup2            | 0      | train
3  | audio_transforms    | Compose           | 0      | train
4  | time_mask_transform | TimeMasking       | 0      | train
5  | freq_mask_transform | FrequencyMasking  | 0      | train
6  | melspec_transform   | MelSpectrogram    | 0      | train
7  | db_transform        | AmplitudeToDB     | 0      | train
8  | backbone            | RexNet            | 7.8 M  | train
9  | global_pool         | GeM               | 1      | train
10 | head                | Linear            | 395 K  | train
11 | big_dropout         | Dropout           | 0      | train
-------------------------------------------------------------------
8.2 M     Tra

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

epoch 0 validation loss 1087.9871631524966
epoch 0 validation ROCAUC score 0.4497594292563805
epoch 0 validation C-MAP score pad 3 0.9539528130949679
epoch 0 validation AP score 0.015523832495229272


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 99, global step 35800: 'val_loss' reached 16.32324 (best 16.29009), saving model to '/data2/Mamba/Project/Kaggle/BirdCLEF-2025/cnn_v3/pytorch/train_bce/epoch=99_step=35800_val_roc_auc=0.968_val_cmap_pad=0.845_val_ap=0.847.ckpt' as top 5


epoch 99 validation loss 16.323242330703035
epoch 99 validation ROCAUC score 0.9679425552102743
epoch 99 validation C-MAP score pad 3 0.8454604318825567
epoch 99 validation AP score 0.8472050944312408


`Trainer.fit` stopped: `max_epochs=100` reached.


: 